In [129]:
library(dplyr)
library(synapser)
library(jsonlite)

synLogin()

full_rna <- synGet(entity='syn25999238')
targets <- synGet(entity='syn12540368')

full_rna <- fromJSON(full_rna$path) %>% as.data.frame
targets <- read.csv(file=targets$path) %>% as.data.frame

Welcome, Matthew Fazza!

NULL

In [130]:
print(colnames(targets))

targets <- 
    targets %>% 
    dplyr::rename_all(tolower) %>%
    dplyr::rename_all(.funs=stringr::str_replace_all, pattern=" ", replacement="_") %>%
    dplyr::filter(validation.study.details != 'not prioritized for experimental validation') %>%
    dplyr::select(ensembl_gene_id, hgnc_symbol)

targets


 [1] "Source"                               
 [2] "Team"                                 
 [3] "Rank"                                 
 [4] "ensembl_gene_id"                      
 [5] "hgnc_symbol"                          
 [6] "Target_Choice_Justification"          
 [7] "Predicted_Therapeutic_Direction"      
 [8] "Data_used_to_support_target_selection"
 [9] "data_synapseID"                       
[10] "study"                                
[11] "input_data"                           
[12] "Validation.study.details"             
[13] "initial_nomination"                   


ensembl_gene_id,hgnc_symbol
<chr>,<chr>
ENSG00000198840,MT-ND3
ENSG00000149187,CELF1
ENSG00000120948,TARDBP
ENSG00000084734,GCKR
ENSG00000154330,PGM5
ENSG00000105618,PRPF31
ENSG00000171492,LRRC8D
ENSG00000163536,SERPINI1
ENSG00000105467,SYNGR4


In [131]:
selected_rna <- full_rna %>% 
    dplyr::inner_join(., targets, by=c('ensembl_gene_id', 'hgnc_symbol')) %>%
    dplyr::select(ensembl_gene_id, hgnc_symbol, tissue, adj_p_val) %>%
    dplyr::mutate(normalized_val = 1-adj_p_val**(1/3)) %>%
    dplyr::group_by(ensembl_gene_id, hgnc_symbol) %>%
    tidyr::nest() %>%
    dplyr::rename(rna=data)
    
    
# selected_rna.shape
# selected_rna = selected_rna[['ensembl_gene_id', 'hgnc_symbol', 'tissue', 'adj_p_val']]
# selected_rna['normalized_value'] = 1-selected_rna['adj_p_val']**(1/3)
# # selected_rna.head()
# selected_rna[selected_rna['ensembl_gene_id']=='ENSG00000003147']

In [132]:
print(selected_rna)
selected_rna$rna[1]

# A tibble: 209 x 3
# Groups:   ensembl_gene_id, hgnc_symbol [209]
   ensembl_gene_id hgnc_symbol rna              
   <chr>           <chr>       <list>           
 1 ENSG00000067057 PFKP        <tibble [28 × 3]>
 2 ENSG00000131711 MAP1B       <tibble [28 × 3]>
 3 ENSG00000162241 SLC25A45    <tibble [28 × 3]>
 4 ENSG00000003147 ICA1        <tibble [28 × 3]>
 5 ENSG00000198743 SLC5A3      <tibble [28 × 3]>
 6 ENSG00000163596 ICA1L       <tibble [28 × 3]>
 7 ENSG00000110492 MDK         <tibble [56 × 3]>
 8 ENSG00000131408 NR1H2       <tibble [28 × 3]>
 9 ENSG00000198690 FAN1        <tibble [28 × 3]>
10 ENSG00000099622 CIRBP       <tibble [28 × 3]>
# … with 199 more rows


tissue,adj_p_val,normalized_val
<chr>,<dbl>,<dbl>
CBE,0.0000000448,0.99644838
TCX,0.1150927390,0.51357490
CBE,0.0000035815,0.98470011
CBE,0.0047919441,0.83140789
TCX,0.1909858789,0.42411767
TCX,0.4631241485,0.22631209
CBE,0.0000000755,0.99577349
TCX,0.0796348578,0.56976962
FP,0.4107797224,0.25663348


In [133]:
proc <- function(x){
    as.data.frame(x) %>%
    dplyr::group_by(tissue) %>% 
    dplyr::summarise(adj_p_val = mean(adj_p_val), normalized_val = mean(normalized_val))
}

selected_rna$rna <- selected_rna$rna %>%
    lapply(., proc)

In [134]:
selected_rna %>%
    jsonlite::toJSON(pretty=2, digits=NA) %>%
    readr::write_lines('../output/prioritized_targets_rna.json')